In [14]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor


In [15]:
f = open('../../dataset/clickbait_dataset/vectorized_roberta20k.json','r')
data = json.load(f)


In [ ]:
X=[]
Y=[]
T=[]
ID=[]
for x in data:
    ID.append(x)
    # T.append(data[x][0])
    Y.append(data[x][0])
    X.append(data[x][1])

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [16]:
X.shape

(19482, 1024)

In [17]:
# put ratio here
test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=0)

In [8]:
print(X_train[0],Y_train[0],file=open("./tmp.txt","w+"))

In [18]:
DTR = DecisionTreeRegressor(max_depth=1)
abr = AdaBoostRegressor(n_estimators=30, base_estimator=DTR ,learning_rate=1)
lr = linear_model.LinearRegression()
gbr_params = {'n_estimators': 1000,'max_depth': 5,'min_samples_split': 5,'learning_rate': 0.01}
gbr = GradientBoostingRegressor(**gbr_params)
rfr = RandomForestRegressor(n_estimators=20, random_state= 0)
rr = Ridge(alpha=0.1,normalize=True)
xgbr_params = {'n_estimators': 1000,'max_depth': 10,'learning_rate': 0.01,'booster': 'gbtree', 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.1, 'reg_lambda': 1}
xgbr = XGBRegressor(objective='reg:squarederror',**xgbr_params)

In [19]:
estimators = [
    ('gbr', gbr),
    ('rr', rr),
    ('rfr', rfr),
    ('abr', abr),
    ('xgbr',xgbr)
]
reg = StackingRegressor(estimators=estimators,final_estimator=linear_model.LinearRegression(),n_jobs=-1)
reg.fit(X_train, Y_train).score(X_test, Y_test)

0.5105425235603611

In [23]:
print("MSE: ",mean_squared_error(Y_test, reg.predict(X_test)))

MSE:  0.03047084040207109


In [21]:
print("MSE for train: ",mean_squared_error(Y_train, reg.predict(X_train)))

MSE for train:  0.018138069477998883


In [22]:
import joblib
joblib.dump(reg, 'stkRobert_20k.pkl')


['stkRobert_20k.pkl']

In [24]:

saved_model = joblib.load('stkRobert_20k.pkl')

print("MSE: ",mean_squared_error(Y_test, saved_model.predict(X_test)))

MSE:  0.03047084040207109


In [25]:
bbb=X_test[0]
aaa=saved_model.predict(bbb.reshape(1,-1))
print(aaa)

[0.32043458]


In [ ]:
# 2k=>0.04236049536066814,,,0.04238977575476603, MSE:  0.042104959037181894
# 2ok pla 0.03721944133053812